In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    balanced_accuracy_score, roc_auc_score, f1_score, precision_score,
    recall_score, classification_report, make_scorer
)


# === LOAD DATA ===
eq5d = pd.read_csv('../rq1/rq1_cleaned_no_ae.csv')
scores = pd.read_excel('../data/Scores 6 Jan 2025_Prescribed_Completed Baseline PROMs.xlsx')
df = pd.read_csv('../rq1/rq1_cleaned_adverse_binary.csv')

# === MERGE PROMs ===
gad7 = scores[scores['promName']=='GAD7'][['SID','Round','total_score']]
gad7_wide = gad7.pivot_table(index='SID', columns='Round', values='total_score', aggfunc='first')
gad7_wide.columns = [f"GAD7_Round{r}" for r in gad7_wide.columns]
gad7_wide = gad7_wide.reset_index()
gad7 = pd.merge(eq5d, gad7_wide, on='SID', how='left')

ins = scores[scores['promName']=='insomniaEfficacyMeasure'][['SID','Round','total_score']]
ins_wide = ins.pivot_table(index='SID', columns='Round', values='total_score', aggfunc='first')
ins_wide.columns = [f"insomniaEfficacyMeasure_Round{r}" for r in ins_wide.columns]
ins_wide = ins_wide.reset_index()



TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [16]:
full = pd.merge(gad7, ins_wide, on='SID', how='left')
full['adverse_binary'] = df['adverse_binary'].astype(int)

# === CLEAN DATA ===
drop_cols = ['SID', 'GAD7_Round2','GAD7_Round3','GAD7_Round4','GAD7_Round5','GAD7_Round6','GAD7_Round7',
             'GAD7_Round8','GAD7_Round9','GAD7_Round10','GAD7_Round11','GAD7_Round12','GAD7_Round13',
             'EQ5D_Round2','EQ5D_Round3','EQ5D_Round4','EQ5D_Round5','EQ5D_Round6',
             'insomniaEfficacyMeasure_Round2','insomniaEfficacyMeasure_Round3','insomniaEfficacyMeasure_Round4',
             'insomniaEfficacyMeasure_Round5','insomniaEfficacyMeasure_Round6','insomniaEfficacyMeasure_Round7',
             'insomniaEfficacyMeasure_Round8','insomniaEfficacyMeasure_Round9','insomniaEfficacyMeasure_Round10',
             'insomniaEfficacyMeasure_Round11','insomniaEfficacyMeasure_Round12','insomniaEfficacyMeasure_Round13','GAD7_Round1_y', 'insomniaEfficacyMeasure_Round1_y'
]


X = full.drop(columns=drop_cols)
y=full['adverse_binary']

data = pd.concat([X, y], axis=1).dropna()
X, y = data.drop(columns='adverse_binary'), data['adverse_binary']
X = X.rename(columns={
    'GAD7_Round1_x': 'GAD7_Round1',
    'insomniaEfficacyMeasure_Round1_x': 'insomniaEfficacyMeasure_Round1'
})

y = data.loc[:, 'adverse_binary'].iloc[:, 0] 
